# Web Scraping

Este script demuestra cómo hacer web scraping usando Playwright para extraer información de productos de una tienda online de ejemplo.

### Requisitos:
- Instalar las dependencias necesarias:
  > pip install pandas requests beautifulsoup4

  

In [ ]:
%pip install pandas requests beautifulsoup4

# Librerías

In [ ]:
import requests
import re
from bs4 import BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random

class GSMArenaScraper:
    """
    Clase para realizar web scraping de la página de fabricantes de GSMArena.

    Atributos:
        url (str): URL de la página de fabricantes.
        brands (list): Lista para almacenar la información de las marcas.
        session (requests.Session): Sesión HTTP para reutilizar conexiones.
        request_count (int): Contador de peticiones realizadas.
        pause_every (int): Número de peticiones después de las cuales se realiza una pausa.
        pause_time_range (tuple): Rango de segundos para pausar (min, max).
    """

    def __init__(self, url='https://www.gsmarena.com/makers.php3', pause_every=10, pause_time_range=(5, 10)):
        """
        Inicializa el scraper.

        Args:
            url (str): URL inicial de fabricantes de GSMArena.
            pause_every (int): Número de peticiones tras las cuales se pausa.
            pause_time_range (tuple): Rango de tiempo aleatorio para pausar.
        """
        self.url = url
        self.brands = []
        self.session = requests.Session()
        self.request_count = 0
        self.pause_every = pause_every
        self.pause_time_range = pause_time_range

    def _check_pause(self):
        """
        Verifica si se debe pausar el scraping según el número de peticiones realizadas.
        Introduce un retraso aleatorio entre peticiones para evitar bloqueos.
        """
        if self.request_count > 0 and self.request_count % self.pause_every == 0:
            wait_time = random.randint(*self.pause_time_range)
            print(f"Se alcanzó {self.request_count} peticiones. Pausando {wait_time} segundos...")
            time.sleep(wait_time)

    def fetch_page(self, url=None):
        """
        Realiza la solicitud HTTP y devuelve el contenido HTML de una página.

        Args:
            url (str, opcional): URL a solicitar. Si no se proporciona, se usa la URL principal.

        Returns:
            str: Contenido HTML de la página.
        """
        if url is None:
            url = self.url
        response = self.session.get(url)
        response.raise_for_status()
        self.request_count += 1
        self._check_pause()
        return response.text

    def parse_brands(self, html):
        """
        Analiza el HTML y extrae información de marcas.

        Args:
            html (str): Contenido HTML de la página de fabricantes.

        Actualiza:
            self.brands: lista de diccionarios con 'name', 'url' y 'devices'.
        """
        try:
            soup = BeautifulSoup(html, 'html.parser')
            makers_container = soup.find('div', class_='st-text')
            brand_links = makers_container.find_all('a')

            for link in brand_links:
                brand_name = link.contents[0].strip()
                brand_url = 'https://www.gsmarena.com/' + link.get('href')
                devices_text = link.find('span').get_text(strip=True)
                devices_number = int(re.search(r'\d+', devices_text).group())
                self.brands.append({
                    'name': brand_name,
                    'url': brand_url,
                    'devices': devices_number
                })
        except Exception as e:
            print(f'Error al analizar el HTML: {e}')

    def scrape(self):
        """
        Método principal para ejecutar el scraping de marcas.

        Returns:
            list: Lista de diccionarios con información de las marcas.
        """
        html = self.fetch_page()
        self.parse_brands(html)
        return self.brands

    def scrape_brand_devices(self, brand_url):
        """
        Extrae la lista de dispositivos de una marca específica.

        Args:
            brand_url (str): URL de la página de la marca.

        Returns:
            list: Lista de diccionarios con 'name' y 'url' de cada dispositivo.
        """
        try:
            html = self.fetch_page(brand_url)
            soup = BeautifulSoup(html, 'html.parser')
            devices = []
            device_links = soup.find_all('div', class_='makers')[0].find_all('a')

            for link in device_links:
                device_name = link.find('strong').get_text(strip=True)
                device_url = 'https://www.gsmarena.com/' + link.get('href')
                devices.append({'name': device_name, 'url': device_url})
            return devices
        except Exception as e:
            print(f'Error al raspar dispositivos: {e}')
            return []

# Uso del scraper
scraper = GSMArenaScraper(pause_every=5, pause_time_range=(5, 10))
brands = scraper.scrape()
for b in brands:
    devices = scraper.scrape_brand_devices(b['url'])
    print(f"Dispositivos de {b['name']}:")
    for d in devices:
        print(f" - {d['name']}: {d['url']}")


HTTPError: 429 Client Error: Too Many Requests for url: https://www.gsmarena.com/makers.php3

In [19]:
scraper